# Movie Recommended System

In [35]:
import pandas as pd
movies = pd.read_csv("./data/movies.csv")
credits = pd.read_csv("./data/credits.csv")

In [36]:
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [37]:
credits.head(1)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


Mergin 2 data on basis of title

In [38]:
movies = movies.merge(credits,on="title")
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


# Removing unwanted feature

In [39]:
# genres, id, keywords, title, overview, cast, crew
movies = movies[["movie_id","title","overview","cast","crew","keywords","genres"]]
movies.head(1)

,movie_id,title,overview,cast,crew,keywords,genres
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam..."


# handling missing and duplicated data

In [40]:
movies.dropna(inplace=True), movies.duplicated().sum()

(None, 0)

# data preprocessing in each column

In [41]:
import ast
def convert(obj):
  res = []
  for  i in ast.literal_eval(obj):
    res.append(i["name"])
  return res

movies["genres"] = movies["genres"].apply(convert)
movies["keywords"] = movies["keywords"].apply(convert)


In [42]:
def convert_cast(obj):
  res = []
  cnt = 0
  for i in ast.literal_eval(obj):
    if cnt < 3:
      res.append(i["name"])
      cnt+=1
    elif cnt > 3:
      break
  return res
movies["cast"] = movies["cast"].apply(convert_cast)

In [43]:
def convert_crew(obj):
  res = [] 
  for i in ast.literal_eval(obj):
    if i["job"] == "Director":
      res.append(i["name"])
      break  
  return res

movies["crew"] = movies["crew"].apply(convert_crew)

In [44]:
movies["overview"] = movies["overview"].apply(lambda x:x.split())
movies.head(1)

,movie_id,title,overview,cast,crew,keywords,genres
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron],"[culture clash, future, space war, space colon...","[Action, Adventure, Fantasy, Science Fiction]"


In [45]:
movies["genres"] = movies["genres"].apply(lambda x:[i.replace(" ","") for i in x])
# movies["crew"] = movies["crew"].apply(lambda x:[i[0].replace(" ","") for i in x])
movies["cast"] = movies["cast"].apply(lambda x:[i.replace(" ","") for i in x])
movies["keywords"] = movies["keywords"].apply(lambda x:[i.replace(" ","") for i in x])

In [46]:
def remove_space():
  missing = []
  cnt = 0
  for i in range(len(movies["crew"])):
    if movies["crew"].iloc[i] == []:
      c = movies.drop(i)
      print("Dropping row = ",c)
      cnt +=1 
# movies["crew"].sample(n=50) 
remove_space()
movies["crew"] = movies["crew"].apply(lambda x:[i.replace(" ","") for i in x])


Dropping row =        movie_id                                     title  \
0        19995                                    Avatar   
1          285  Pirates of the Caribbean: At World's End   
2       206647                                   Spectre   
3        49026                     The Dark Knight Rises   
4        49529                               John Carter   
...        ...                                       ...   
4804      9367                               El Mariachi   
4805     72766                                 Newlyweds   
4806    231617                 Signed, Sealed, Delivered   
4807    126186                          Shanghai Calling   
4808     25975                         My Date with Drew   

                                               overview  \
0     [In, the, 22nd, century,, a, paraplegic, Marin...   
1     [Captain, Barbossa,, long, believed, to, be, d...   
2     [A, cryptic, message, from, Bond’s, past, send...   
3     [Following, the, deat

In [47]:
movies["tags"] = movies["overview"] + movies["cast"] + movies["crew"] + movies["genres"] + movies["keywords"]

In [48]:
movies.head(1)

,movie_id,title,overview,cast,crew,keywords,genres,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron],"[cultureclash, future, spacewar, spacecolony, ...","[Action, Adventure, Fantasy, ScienceFiction]","[In, the, 22nd, century,, a, paraplegic, Marin..."


# New data frame

In [49]:
new_df = movies[["movie_id","tags","title"]]

In [50]:
new_df["tags"] = new_df["tags"].apply(lambda x:" ".join(x))
new_df["tags"] = new_df["tags"].apply(lambda x:x.lower())

/tmp/ipykernel_2693/1914434289.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["tags"] = new_df["tags"].apply(lambda x:" ".join(x))
/tmp/ipykernel_2693/1914434289.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["tags"] = new_df["tags"].apply(lambda x:x.lower())


In [51]:
new_df["tags"][0]

'in the 22nd century, a paraplegic marine is dispatched to the moon pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. samworthington zoesaldana sigourneyweaver jamescameron action adventure fantasy sciencefiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d'

# Stemming

In [52]:
from nltk import PorterStemmer
ps = PorterStemmer()
def stem(text):
  y = []
  for i in text.split():
    y.append(ps.stem(i))
  return " ".join(y)


In [53]:
new_df["tags"] = new_df["tags"].apply(stem)

/tmp/ipykernel_2693/825579503.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["tags"] = new_df["tags"].apply(stem)


In [54]:
new_df["tags"][0]

'in the 22nd century, a parapleg marin is dispatch to the moon pandora on a uniqu mission, but becom torn between follow order and protect an alien civilization. samworthington zoesaldana sigourneyweav jamescameron action adventur fantasi sciencefict cultureclash futur spacewar spacecoloni societi spacetravel futurist romanc space alien tribe alienplanet cgi marin soldier battl loveaffair antiwar powerrel mindandsoul 3d'

# Vectorize the text

In [55]:
# Vectorize the text
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words="english")
vectors = cv.fit_transform(new_df["tags"]).toarray()
vectors.shape
cv.get_feature_names_out()


array(['000', '007', '10', ..., 'zone', 'zoo', 'zooeydeschanel'],
      dtype=object)

In [64]:
vectors.shape
from sklearn.metrics.pairwise import cosine_similarity
simlarity = cosine_similarity(vectors)

# recommend movie

In [67]:
def recommend(movie):
  movie_index = new_df[new_df["title"] == movie].index[0]
  distances = simlarity[movie_index]
  movie_list = sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]
  for i in movie_list:
    tit = new_df.iloc[i[0]].title
    print("Recommend Movie = ",tit)
  return


In [81]:
recommend("Batman Begins")

Recommend Movie =  The Dark Knight
Recommend Movie =  Batman
Recommend Movie =  Batman
Recommend Movie =  The Dark Knight Rises
Recommend Movie =  10th & Wolf


In [90]:
import pickle as pk
pk.dump(new_df,open("./dump/data.pkl","wb"),protocol=4)

In [91]:
pk.dump(simlarity,open("./dump/similarity.pkl","wb"),protocol=4)